<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Piecewise Parabolic Method for Variable Reconstruction

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:
* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).
* **(Recommended)** [Colella and Woodward. The Piecewise Parabolic Method (PPM) for gas-dynamical simulations (1984)](https://www.sciencedirect.com/science/article/pii/0021999184901438).
* **(Recommended)** [Martı́ and Müller. Extension of the Piecewise Parabolic Method to One-Dimensional Relativistic Hydrodynamics (1996)](https://www.sciencedirect.com/science/article/pii/S0021999196900017).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1. [**The `ppm` function**](#ppm)
    1. [*Function Arguments*](#fargs)
        1. [Inputs](#inputs)
        1. [Outputs](#outputs)
    1. [*Function Behavior*](#behav)
        1. [The `ppm_Ur_Ul` function](#ppm_Ur_Ul)
        1. [The `slope_limit` function](#slope_limit)
        1. [The `compute_UrUl_onevar` function](#compute_UrUl_onevar)
        1. [The `steepen_rhor_rhol` function](#steepen_rhor_rhol)
        1. [The `shock_detection_ftilde` function](#shock_detection_ftilde)
        1. [The `flatten_and_monotonize_Ur_and_Ul` function](#flatten_and_monotonize_Ur_and_Ul)
1. [**The `ppm_no_rho_P` function**](#ppm_no_rho_P)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The `ppm` and `ppm_no_rho_P` functions reconstruct variables onto a face. The actual functions are direction-agnostic, and the face being reconstructed is purely dependent on the stencils being passed in. All the stencils for these functions are of length 6. To reconstruct to the $i-\frac{1}{2}$ face, pass stencils from $i-3$ to $i+2$. To reconstruct to the $i+\frac{1}{2}$ face, pass stencils from $i-2$ to $i+3$.

<a id='ppm'></a>

# The `ppm` function \[Back to [top](#toc)\]

$$\label{ppm}$$

<a id='args'></a>

## Function Arguments \[Back to [top](#toc)\]
$$\label{args}$$


First, let's look at the arguments of `ppm`:
```
void ppm
      const double rho[6],
      const double pressure[6],
      const double var_data[][6],
      const int num_vars,
      const double v_flux_dirn[6],
      const double Gamma_eff,
      double *restrict rhor,
      double *restrict rhol,
      double *restrict pressr,
      double *restrict pressl,
      double *restrict var_datar,
      double *restrict var_datal)
```

<a id='inputs'></a>

### Inputs
$$\label{inputs}$$

`rho`, `pressure`, and `var_data` contain stencils for $\rho_b$, $P$, and the other variables to be reconstructed. The stencil length is 6 because the function reconstructs the left and right faces, each of which requires 5 points. `var_data` is a 2D array containing stencils for all the other variables to be reconstructed. The first array argument will loop from 0 to `num_vars`, so the passed array should have dimensions `var_data[num_vars][6]`. `v_flux_dirn` is the velocity $v^i$ in the direction of the reconstruction. For example, the quantity $v^x$ would be passed to reconstruct in the $x$ direction. Finally, `Gamma_eff` is the effective gamma for the reconstruction. This is computed in `IllinoisGRMHD` with
$$
  \Gamma_\mathrm{th} + \frac{P_\mathrm{cold}}{P}(\Gamma_\mathrm{ppoly}[\rho] - \Gamma_\mathrm{th})
$$

<a id='outputs'></a>

### Outputs
$$\label{outputs}$$

`rhor`, `pressr`, and `var_datar` contain the reconstructed right faces for $\rho_b$, $P$, and the other variables. `rhol`, `pressl`, and `var_datal` contain the reconstructed left faces for $\rho_b$, $P$, and the other variables. This function always reconstructs $\rho_b$ and $P$. $\rho_b$ is separate because an additional steepening function applies to the $\rho_b$ reconstruction.

<a id='behav'></a>

## Function Behavior
$$\label{behav}$$

The actual `ppm` function is quite short. It calls the `ppm_Ur_Ul` function twice, once for each side of the face. For a concrete example, let us assume we want to construct the face at $i-\frac{1}{2}$. This means we are passing a stencil from $i-3$ to $i+2$. Then, the desired return values are
$$
U_r = U(i-\frac{1}{2}+\epsilon) \\
U_l = U(i-\frac{1}{2}-\epsilon)
$$
where $U$ represents the variable being reconstructed. The first `ppm_Ur_Ul` call computes
$$
U^R(i) = U\left(i+\frac{1}{2}\right) \\
U^L(i) = U\left(i-\frac{1}{2}\right)
$$
using the stencil centered around $U(i)$. Thus, `ppm` passes the last 5 points of the stencil to `ppm_Ur_Ul`. Since we are constructing the $i-\frac{1}{2}$ face, reconstructing from $i$ gives us the right side of the face:
$$
U_r = U(i-\frac{1}{2}+\epsilon) = U^{L}(i)
$$
Then, we pass the first 5 stencil points to `ppm_Ur_Ul`, which computes
$$
U^(i-1) = U\left(i-\frac{1}{2}\right) \\
U^L(i-1) = U\left(i-\frac{3}{2}\right)
$$
Reconstructing from $i-1$ gives us the left side of the face:
$$
U_l = U(i-\frac{1}{2}-\epsilon) = U^{R}(i-1)
$$

None of the internal function changes for reconstructing to $i+\frac{1}{2}$, as this is purely dependent on which direction the input stencils are biased.

<a id='ppm_Ur_Ul'></a>

### The `ppm_Ur_Ul` Function \[Back to [top](#toc)\]

$$\label{ppm_Ur_Ul}$$

The core of the reconstruction method is the `ppm_Ur_Ul` function. This function actually computes the right and left faces for a point. The arguments are nearly identical to `ppm`
```
void ppm_Ur_Ul(
      const double rho[5],
      const double pressure[5],
      const double var_data[][5],
      const int num_vars,
      const double v_flux_dirn[5],
      const double Gamma_eff, // Gamma_eff = (partial P / partial rho0)_s /(P/rho0)
      double *restrict rhor,
      double *restrict rhol,
      double *restrict pressr,
      double *restrict pressl,
      double *restrict varsr,
      double *restrict varsl)
```
except that the stencils are 5 instead of 6. This stencil is centered on the point $i$ being reconstructed. The reconstruction process is split into several steps, but the general flow is, for each variable $U$,
1. Compute $U^R$ and $U^L$ using `compute_UrUl_onevar`
1. For $\rho_b$, call `steepen_rhor_rhol`
1. Further adjust $U^R$ and $U^L$ using `flatten_and_monotonize_Ur_and_Ul`

We will describe these functions in more detail, but these simple steps are the entire process for computing $U^R$ and $U^L$.

<a id='slope_limit'></a>

### The `slope_limit` Function \[Back to [top](#toc)\]

$$\label{slope_limit}$$

The `slope_limit` function
```
double slope_limit(
      const double dU,
      const double dUp1)
```
returns the slope-limited $\Delta U$. This function is used by several functions inside the PPM routine. If we have a stencil for the variable $U$, then the call would take the form
$$
dU_i = \mathrm{slope\_limit}(U_{i} - U_{i-1}, U_{i+1} - U_i)
$$
This function is based on Eq. 60 of [Martı́ and Müller](https://www.sciencedirect.com/science/article/pii/S0021999196900017) (note the factor of 2 missing in the $|a_{j+1} - a_{j}|$ term). Defining
$$
\Delta U_i = U_{i} - U_{i-1} \\
\Delta U_{i+1} = U_{i+1} - U_i
$$
`slope_limit` returns 0 if $dU_i$ and $dU_{i+1}$ have different signs. If not, it first computes the average
$$
\Delta U_\mathrm{avg} = \frac{1}{2}\left(\Delta U_i + \Delta U_{i+1}\right)
$$
and then sets the sign $\sigma$ of the slope using
$$
\sigma = (0.0 < \Delta U_\mathrm{avg}) - (\Delta U_\mathrm{avg} < 0.0)
$$
This ensures that $\sigma$ matches the signs of $\Delta U_i$ and $\Delta U_{i+1}$. Finally, the slope is determined by returning
$$
\mathrm{MIN} \left( \Delta U_\mathrm{avg}, 2U_i, 2U_{i+1} \right)
$$

<a id='compute_UrUl_onevar'></a>

### The `compute_UrUl_onevar` Function \[Back to [top](#toc)\]

$$\label{compute_UrUl_onevar}$$

The `compute_UrUl_onevar` function
```
void compute_UrUl_onevar(
      const double U[5],
      double *restrict Ur,
      double *restrict Ul)
```
computes the $U^R$ and $U^L$ values. It first computes the derivatives at $i-1$, $i$, and $i+1$ using the `slope_limit` function

$$
\begin{align}
dU_{i-1} &= \mathrm{slope\_limit}(U_{i-1} - U_{i-2}, U_{i} - U_{i-1}) \\
dU_i     &= \mathrm{slope\_limit}(U_{i} - U_{i-1}, U_{i+1} - U_i) \\
dU_{i+1} &= \mathrm{slope\_limit}(U_{i+1} - U_{i}, U_{i+2} - U_{i+1})
\end{align}
$$

Then,
$$
U^R = \frac{1}{2}\left(U_{i+1} + U_i\right) + \frac{1}{6}\left(dU_i - dU_{i+1}\right) \\
U^L = \frac{1}{2}\left(U_i + U_{i-1}\right) + \frac{1}{6}\left(dU_{i-1} - dU_i\right) \\
$$

<a id='steepen_rhor_rhol'></a>

### The `steepen_rhor_rhol` Function \[Back to [top](#toc)\]

$$\label{steepen_rhor_rhol}$$

Reconstruction of $\rho_b$ gets special treatment (TODO: reason)). This is handled by the `steepen_rhor_rhol` function
```
void steepen_rhor_rhol(
      const double rho[5],
      const double P[5],
      const double Gamma_eff,
      double *restrict rhor,
      double *restrict rhol)
```
For clarity, the centerpoint of the stencil (length 5) is denoted as $i$, and the stencil ranges from $i-2$ to $i+2$. The function first computes the centered differences

$$
\begin{align}
\Delta\rho_b &= \frac{\rho_b^{i+1} - \rho_b^{i-1}}{2} \\
\Delta^2\rho_b^{i-1} &= \rho_b^{i} - 2\rho_b^{i-1} + \rho_b^{i-2} \\
\Delta^2\rho_b^{i+1} &= \rho_b^{i+2} - 2\rho_b^{i+1} + \rho_b^{i}
\end{align}
$$

and the minimum adjacent point
$$
\rho^{\min}_b = \mathrm{MIN}\left( \rho_b^{i-1}, \rho_b^{i+1} \right)
$$

We finally use the $\Gamma_\mathrm{eff}$ argument here. How it is computed is up to the user, but it is defined as
$$
\Gamma_\mathrm{eff} = \frac{\rho_b}{P}\frac{\partial P}{\partial \rho_b}
$$

$$
{\rm contact_discontinuity_check} = \frac{\Gamma_\mathrm{eff}}{10} \left| \rho_b^{i+1}-\rho_b^{i-1} \right|\mathrm{MIN}\left( P^{i-1}, P^{i+1} \right)
 - \rho^{\min}_b\left| P^{i+1}-P^{i-1} \right|
$$

$$
{\rm second_deriv_check} = -\Delta^2\rho_b^{i-1}*\Delta^2\rho_b^{i+1}
$$

$$
{\rm relative_change_check} = 2\left| \Delta\rho_b \right| - \frac{\rho^{\min}_b}{100}
$$

if all >=0:

```
    const double slope_limited_drho_m1 = slope_limit(rho[MINUS1] - rho[MINUS2], rho[PLUS_0] - rho[MINUS1]);
    const double slope_limited_drho_p1 = slope_limit(rho[PLUS_1] - rho[PLUS_0], rho[PLUS_2] - rho[PLUS_1]);
```


```
#include "reconstruction.h"
  
// standard Colella-Woodward parameters:
//    K0 = 0.1d0, eta1 = 20.0, eta2 = 0.05, epsilon = 0.01d0

    double eta_tilde=0.0;
    if (fabs(d1rho_b) > 0.0) {
      eta_tilde = -(1.0/6.0)*(d2rho_b_p1 - d2rho_b_m1)/(2.0*d1rho_b);
    }
    const double eta = MAX(0.0,MIN(ETA1*(eta_tilde - ETA2),1.0));
    // Next compute Urp1 and Ul for RHOB, using the MC prescription:
    // Ur_p1 = U_p1   - 0.5*slope_lim_dU_p1
    const double rho_br_mc_p1 = rho[PLUS_1] - 0.5*slope_limited_drho_p1;
    // Ul = U_m1 + 0.5*slope_lim_dU_m1
    // Based on this line of code, Ur[index] = a_j - \delta_m a_j / 2. (cf. Eq. 65 in Marti & Muller's "PPM Method for 1D Relativistic Hydro." paper)
    //    So: Ur[indexp1] = a_{j+1} - \delta_m a_{j+1} / 2. This is why we have rho_br_mc[indexp1]
    const double rho_bl_mc    = rho[MINUS1] + 0.5*slope_limited_drho_m1;

    *rhol = (*rhol)*(1.0 - eta) + rho_bl_mc*eta;
    *rhor = (*rhor)*(1.0 - eta) + rho_br_mc_p1*eta;

  }
}
```

<a id='shock_detection_ftilde'></a>

### The `shock_detection_ftilde` Function \[Back to [top](#toc)\]

$$\label{shock_detection_ftilde}$$

The `shock_detection_ftilde` function
```
double shock_detection_ftilde(
      const double P[5],
      const double v_flux_dirn[5])
```
returns $\tilde{f}$, which is used to flatten $U^R$ and $U^L$ in the presence of shocks. It requires the `pressure` and `v_flux_dirn` arguments which are passed to `ppm_Ur_Ul`. For this section, we refer to `v_flux_dirn` as $v$.

In addition to computing $\tilde{f}$, we also need to determine if there is a shock. To begin, we compute

$$
\begin{align}
\Delta P_1 &= P_{i+1} - P_{i-1} \\
\Delta P_2 &= P_{i+2} - P_{i-2}
\end{align}
$$

and

$$
\begin{align}
\mathrm{avg}_1 &= \frac{1}{2} \left( P_{i-1} + P_{i+1} \right) \\
\mathrm{avg}_2 &= \frac{1}{2} \left( P_{i-2} + P_{i+2} \right)
\end{align}
$$

We use these quantities determine whether or not there is a shock. If
$$
\frac{\Delta P_1}{\mathrm{avg}_1} < 1.5\times 10^{-15}
$$
then there is no shock. Otherwise, if the above is false and
$$
\frac{\Delta P_2}{\mathrm{avg}_2} < 1.5\times 10^{-15}
$$
is true, then there may be a shock. To find out, we compute

$$
\begin{align}
q_1 &= 10\left( \frac{\Delta P_1}{\Delta P_2} -0.75 \right) \\
q_2 &= \frac{|\Delta P_1|}{\mathrm{MIN}(P_{i-1},P_{i+1})}
\end{align}
$$

Using these, we can compute

$$
w_\mathrm{shock} = \left( q_2 > 0.33\ \mathrm{AND}\ q_2(v_{i-1} - v_{i+1}) > 0.0 \right)
$$
If $w_\mathrm{shock} == 1$, then we are inside a shock. Finally, we return
$$
\mathrm{MIN}\left( 1, w_\mathrm{shock} \mathrm{MAX}(0.0, q_1) \right)
$$
If we aren't in a shock ($w_\mathrm{shock} == 0$), then this returns 0. If we are in a shock ($w_{\rm shock} == 1$), then this returns the minimum of 1 and $q_1$, with the caveat that $q_1 > 0$. If $q_1 < 0$, it will always return 1.

<a id='flatten_and_monotonize_Ur_and_Ul'></a>

### The `flatten_and_monotonize_Ur_and_Ul` Function \[Back to [top](#toc)\]

$$\label{flatten_and_monotonize_Ur_and_Ul}$$

The `flatten_and_monotonize_Ur_and_Ul` function
```
void flatten_and_monotonize_Ur_and_Ul(
      const double U,
      const double ftilde,
      double *restrict Ur,
      double *restrict Ul)
```
flattens the function in the case of shocks or steep gradients and then monotonizes the variables. For the variable $U$, we flatten the face values with

$$
\begin{align}
U^R &= \tilde{f}U + U^R (1 - \tilde{f}) \\
U^L &= \tilde{f}U + U^L (1 - \tilde{f})
\end{align}
$$

Then, we monotonize using the intermediate variables

$$
\begin{align}
\Delta U &= U^R - U^L \\
U_\mathrm{tmp} &= \Delta U \left(U - \frac{1}{2}\left(U^R - U^L \right)\right)
\end{align}
$$

There are three special cases we consider. First, if
$$
\left(U^R - U\right)\left(U - U^L\right) \le 0.0
$$
then
$$
U^R = U^L = U
$$

Second, if
$$
U_\mathrm{tmp} > \frac{1}{6}\left(\Delta U \right)^2
$$
then
$$
U^L = 3U - 2U^R
$$

Finally, if
$$
U_\mathrm{tmp} < -\frac{1}{6}\left(\Delta U \right)^2
$$
then
$$
U^R = 3U - 2U^L
$$

<a id='ppm_no_rho_P'></a>

# The `ppm_no_rho_P` function \[Back to [top](#toc)\]

$$\label{ppm_no_rho_P}$$

In addition to `ppm`, we also provide the `ppm_no_rho_P` function. This function is nearly identical to `ppm`, but it does not reconstruct the baryonic density or pressure. We choose to provide two functions to avoid having many `if` statements within the functions. For example, the function `steepen_rhor_rhol` is only called if $\rho_b$ is being reconstructed, so including $\rho_b$ in the `var_data` array would require another argument to tell the code which index contained $\rho_b$ and a check on every reconstruction at every point whether the variable being reconstructed is $\rho_b$. Similarly, $P$ is required regardless of whether it is being reconstructed or not, as it is needed to compute $\tilde{f}$. We could add $P$ to `var_data` whenever we actually want to reconstruct it, but that would be duplicating the stencils. For these reasons, we provide two different functions.

The primary use for this function is reconstructing variables that will get overwritten during the `ppm`. For example, reconstructing the velocities $\left(v^R\right)^L$ and $\left(v^R\right)^R$ can cause a race condition with the reconstruction of $v^R$ from $v$. If the $v^R$ computed from $v$ is copied into a temporary variable which serves as input for reconstructing $\left(v^R\right)^L$ and $\left(v^R\right)^R$, there would not be a conflict. However, that is a lot of duplicated memory, so many codes use $v^R$ as both output and input.

As always, let's first look at the arguments of `ppm_no_rho_P`:
```
void ppm_Ur_Ul_no_rho_P(
      const double pressure[5],
      const double var_data[][5],
      const int num_vars,
      const double v_flux_dirn[5],
      const double Gamma_eff,
      double *restrict varsr,
      double *restrict varsl);

```
These arguments are nearly identical to `ppm`. `pressure` contains the pressure stencil for computing $\tilde{f}$. `var_data` is a 2D array containing stencils for all the variables to be reconstructed. The first array argument will loop from 0 to `num_vars`, so the passed array should have dimensions `var_data[num_vars][6]`. The stencil length is 6 because each reconstruction requires 5 points, and we are reconstructing both the left and right faces. `v_flux_dirn` is the velocity $v$ in the direction of the reconstruction. I.e., the quantity $v^x$ would be passed to reconstruct in the $x$ direction. Finally, `Gamma_eff` is the effective gamma for the reconstruction. This is computed in `IllinoisGRMHD` with
$$
  \Gamma_{th} + \frac{P_{\rm cold}}{P}(\Gamma_{\rm ppoly}[\rho] - \Gamma_{th})
$$

The remaining arguments are the outputs for the right and left values. `var_datar` and `var_datal` are arrays of length `num_vars`.

Other than removing the variables for reconstructing $\rho_b$ and $P$, `ppm` and `ppm_no_rho_P` are identical, as are the internal functions `ppm_Ur_Ul` and `ppm_Ur_Ul_no_rho_P`.

<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
!rm -f Tut*.out Tut*.aux Tut*.log